In [1]:
import joblib
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm
import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, Subset
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import KFold

loaded_data = joblib.load("train_data.pkl")

targets, data, ids = loaded_data

# 打印某个人的具体信息

for i in tqdm(range(277)):
    if ids[i] == "03d92c9f6f8a":
        print(f"Target: {targets[i]}")
        print(f"{data[i]}")
        print(type(targets[i]))
        print(type(data[i]))

100%|██████████| 277/277 [00:00<00:00, 31187.35it/s]

Target: [(5928, 13524), (23220, 30276), (40668, 47952), (75756, 82800), (178464, 186564), (196260, 203844), (230820, 241872), (248124, 255060)]
           anglez      enmo      step
0       38.906250  0.080322       0.0
1       29.375000  0.075195       1.0
2       37.218750  0.179077       2.0
3       46.937500  0.092224       3.0
4       60.500000  0.034210       4.0
...           ...       ...       ...
724135 -12.062500  0.038208  724135.0
724136 -15.914062  0.025406  724136.0
724137 -10.859375  0.028107  724137.0
724138  -8.531250  0.027405  724138.0
724139  -9.273438  0.032013  724139.0

[724140 rows x 3 columns]
<class 'list'>
<class 'pandas.core.frame.DataFrame'>


In [2]:
class CFG:
    window_overlap = 12
    window_size = 12
    train_ratio = 0.80
    batch_size = 32
    input_dim = 2
    embed_dim = 32
    num_classes = 4 
    num_layers = 8
    nhead = 4
    dim_feedforward = 64
    learning_rate = 1e-3
    epochs = 300
    train_record_steps = 140
    test_record_steps = 20
    dropout = 0.4
    num_person = 300
    scheduler_step_size = 20
    scheduler_gamma = 0.8
    fold = 8
    weight_decay = 0.001
    model_dir = "./models"

In [3]:
# 生成放入模型的数据，假设使用30分钟作为一个窗口，那么需要找的为形状（360，2）的矩阵

train_events = pd.read_csv("./train_events.csv")
num_person = len(train_events["series_id"].unique())
print("The number of total persons is:", num_person)

# 用来存储模型的输入与标签
X = []
y = []

for i in tqdm(range(min(num_person, CFG.num_person))):

    df = data[i]
    pair_list = targets[i]
    sleep_id = ids[i]

    # 先提取出开始睡觉和结束睡觉的时间
    onset_steps = []
    wakeup_steps = []
    for j in range(len(pair_list)):
        onset_steps.append(pair_list[j][0])
        wakeup_steps.append(pair_list[j][1])
    # print("length of onset list:", len(onset_steps))
    # print("length of wakeup list:", len(wakeup_steps))
    # print(onset_steps)
    # print(wakeup_steps)

    # 中间没有明确时间点的区间全部都不要
    empty_spaces = []
    events = train_events[train_events["series_id"] == sleep_id]
    last_events = 0
    mark = 0 
    for (idx, row) in events.iterrows():
        if np.isnan(row["step"]):
            mark = 1
        else:
            if row["step"] > last_events and mark == 0:
                last_events = row["step"]
            elif row["step"] > last_events and mark == 1:
                empty_spaces.append((int(last_events + CFG.window_size), int(row["step"] - CFG.window_size)))
                last_events = row["step"]
                mark = 0

    # print(empty_spaces)

    # 为训练数据设置label，如果区间中没有任何事件，标签为0，如果有开始睡觉事件，标签为1，如果有结束睡觉事件，标签为2
    labels = []
    for j in range(0, len(df), CFG.window_overlap):
        start = j
        if j + CFG.window_size >= len(df):
            break
        else:
            end = j + CFG.window_size

        flag_empty = 0
        for k in empty_spaces:
            if (k[0] <= start <= k[1]) or (k[0] <= end <= k[1]):
                flag_empty = 1
                break
            else:
                continue

        flag_sleep = 0
        for k in range(len(onset_steps)):
            if (onset_steps[k] <= start < end <= wakeup_steps[k]):
                flag_sleep = 1
                break
            else:
                continue


        chunk = df.iloc[start:end][["anglez", "enmo"]]
        chunk = chunk.to_numpy()

        if flag_empty == 1:
            label = 4
        elif any(start <= num <= end for num in wakeup_steps):
            label = 3
        elif any(start <= num <= end for num in onset_steps):
            label = 2
        elif flag_sleep == 1:
            label = 1
        else:
            label = 0
        labels.append(label)

        if label != 4:
            X.append(chunk)
            y.append(label)

    element_counts = Counter(labels)

    # for element, count in element_counts.items():
    #     print(f"Element {element} occurs {count} times")

print(len(X), len(y))

The number of total persons is: 277


  1%|          | 3/277 [01:04<1:37:27, 21.34s/it]


KeyboardInterrupt: 

In [ ]:
X_xingzhe = []
y_xingzhe = []
X_shuizhe = []
y_shuizhe = []
X_onset = []
y_onset = []
X_wakeup = []
y_wakeup = []

for i in tqdm(range(len(X))):
    xi = X[i]
    yi = y[i]
    if yi == 0:
        X_xingzhe.append(xi)
        y_xingzhe.append(yi)
    elif yi == 1:
        X_shuizhe.append(xi)
        y_shuizhe.append(yi)
    elif yi == 2:
        X_onset.append(xi)
        y_onset.append(yi)
    elif yi == 3:
        X_wakeup.append(xi)
        y_wakeup.append(yi)

print(len(X_xingzhe), len(X_shuizhe), len(X_onset), len(X_wakeup))

X_xingzhe = X_xingzhe[:max(len(X_onset), len(X_wakeup))]
y_xingzhe = y_xingzhe[:max(len(X_onset), len(X_wakeup))]
X_shuizhe = X_shuizhe[:max(len(X_onset), len(X_wakeup))]
y_shuizhe = y_shuizhe[:max(len(X_onset), len(X_wakeup))]

X = X_xingzhe + X_shuizhe + X_onset + X_wakeup
y = y_xingzhe + y_shuizhe + y_onset + y_wakeup

print(len(X), len(y))

In [ ]:
class MyDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        inp = self.inputs[index]
        output = self.outputs[index]

        input_tensor = torch.tensor(inp, dtype=torch.float32)
        output_tensor = torch.tensor(output, dtype=torch.long)

        return input_tensor, output_tensor


dataset = MyDataset(X, y)
print(len(dataset))

# train_size = int(CFG.train_ratio * len(dataset))
# test_size = len(dataset) - train_size

# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=True)
# print(len(train_loader), len(test_loader))

In [ ]:
class TransformerModel(nn.Module):
    def __init__(
        self,
        input_dim,
        embed_dim,
        num_classes=3,
        num_layers=8,
        nhead=4,
        dim_feedforward=64,
        dropout=0.4
    ):

        super(TransformerModel, self).__init__()

        self.conv1d = nn.Conv1d(
            in_channels=input_dim, 
            out_channels=embed_dim, 
            kernel_size=3, 
            padding=1
        )
        self.embed_layer = nn.Linear(embed_dim, embed_dim)
        self.layernorm = nn.LayerNorm(embed_dim)

        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=nhead,
                dim_feedforward=dim_feedforward,
                dropout=dropout
            ),
            num_layers=num_layers
        )

        self.classification = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)

        x = self.embed_layer(x)
        x = self.layernorm(x)
        x = self.encoder(x.permute(1, 0, 2))
        # print(x.shape)
        x = x[-1]
        x = self.classification(x)
        return x

# 示例
model = TransformerModel(input_dim=2, embed_dim=32)
input_data = torch.rand(3, 360, 2)
output = model(input_data)
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")
print(output.shape)

In [ ]:
model = TransformerModel(
    input_dim=CFG.input_dim,
    embed_dim=CFG.embed_dim,
    num_classes=CFG.num_classes,
    num_layers=CFG.num_layers,
    nhead=CFG.nhead,
    dim_feedforward=CFG.dim_feedforward,
    dropout=CFG.dropout
)

criterion = nn.CrossEntropyLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

In [ ]:
train_losses = []
train_step_losses = []
test_losses = []
test_step_losses = []
learning_rates = []

kf = KFold(n_splits=CFG.fold, shuffle=True, random_state=2023)

for fold, (train_index, test_index) in enumerate(kf.split(dataset)):
    train_subset = Subset(dataset, train_index)
    test_subset = Subset(dataset, test_index)

    train_loader = DataLoader(train_subset, batch_size=CFG.batch_size, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=CFG.batch_size)

    train_model = model.to(device=device)
    train_model.train()

    # init_lr = CFG.learning_rate

    # if fold == 0:
    #     optimizer = torch.optim.Adam(
    #         model.parameters(),
    #         lr=init_lr,
    #         weight_decay=CFG.weight_decay
    #     )
    # else:
    #     # current_learning_rate = optimizer.param_groups[0]["lr"]
    #     # print("current learning rate is:", current_learning_rate)
    #     optimizer = torch.optim.Adam(
    #         model.parameters(),
    #         lr=init_lr / (1.25 ** fold),
    #         weight_decay=CFG.weight_decay
    #     )

    optimizer = torch.optim.Adam(train_model.parameters(), lr=CFG.learning_rate)
    scheduler = StepLR(optimizer, step_size=CFG.scheduler_step_size, gamma=CFG.scheduler_gamma)

    min_loss = 10000.0

    for epoch in range(1, CFG.epochs + 1):
        train_loss = 0.0
        test_loss = 0.0
        train_step_loss = 0.0
        test_step_loss = 0.0


        learning_rate = optimizer.param_groups[0]["lr"]
        learning_rates.append(learning_rate)

        for batch_idx, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
            optimizer.zero_grad()

            data = data.to(device)
            target = target.to(device)
            output = train_model(data)

            loss = criterion(output, target)
            train_loss += loss.item()
            train_step_loss += loss.item()

            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % CFG.train_record_steps == 0:
                train_step_loss /= CFG.train_record_steps
                train_step_losses.append(train_step_loss)
                print(f"Fold: {fold + 1}, Epoch: {epoch + 1}, Step: {(batch_idx + 1) * CFG.train_record_steps}, Train Loss These Steps: {train_step_loss}")
                train_step_loss = 0.0

        scheduler.step()

        with torch.no_grad():
            for batch_idx, (data, target) in tqdm(enumerate(test_loader), total=len(test_loader)):
                data = data.to(device)
                target = target.to(device)
                output = train_model(data)

                loss = criterion(output, target)
                test_loss += loss.item()
                test_step_loss += loss.item()

                if (batch_idx + 1) % CFG.test_record_steps == 0:
                    test_step_loss /= CFG.test_record_steps
                    test_step_losses.append(test_step_loss)
                    print(f"Fold: {fold + 1}, Epoch: {epoch + 1}, Step: {(batch_idx + 1) * CFG.test_record_steps}, Test Loss These Steps: {test_step_loss}")
                    test_step_loss = 0.0

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        test_loss /= len(test_loader)
        test_losses.append(test_loss)

        if test_loss < min_loss:
            min_loss = test_loss
            torch.save(train_model.state_dict(), f"{CFG.model_dir}/model_{fold + 1}.pth")


        print(f"Fold: {fold + 1}, Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss}")

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train & Test Loss")
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_step_losses, label="Train Step Loss")
plt.plot(test_step_losses, label="Test Step Loss")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Train & Test Step Loss")
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(learning_rates)
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")

plt.savefig("plot.png")

plt.tight_layout()
plt.show()